In [1]:
import mysql.connector
from dbconfig import db_config

import pandas as pd
import numpy as np

In [2]:
mydb = mysql.connector.connect(
        host = db_config["host"],
        user = db_config["user"],
        passwd = db_config["password"],
        database = db_config["database"]
    )
mycursor = mydb.cursor()

In [3]:
mycursor.execute("SELECT * FROM Stocks")
aapl = mycursor.fetchall()
sequence = mycursor.column_names

In [4]:
df = pd.DataFrame(aapl, columns = sequence).set_index('date').drop(['symbol', 'high', 'low', 'open'], axis = 1)
df = df.fillna('ffill').fillna('bfill')
df['close'] = pd.to_numeric(df['close'])
df

,close,volume
date,,
2015-06-01,133.31,33166322
2015-06-02,131.26,35078387
2015-06-03,135.94,32034911
2015-06-04,131.78,38463484
2015-06-05,131.58,37314004
...,...,...
2020-05-22,330.48,20958094
2020-05-26,329.79,32870796
2020-05-27,329.09,28621405


#### RSI - 5 day

In [5]:
def MA(df, col, n=5):
    '''Calculate an n-period Moving Average of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col -  specified column to calculate moving average
    n - moving average period
    
    Returns:
    df - dataframe with moving average column
    '''
    df[str(col)+'_MA_'+str(n)] = df[col].rolling(n).mean()
    
    return df

In [6]:
def RSI(df, col='close', n=5):
    '''Calculate an n-period RSI indicator of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col -  specified column to calculate RSI
    n - RSI period
    
    Returns:
    df - dataframe with RSI column
    '''
    df = df.copy()
    df['return'] = df[col].diff()
    df['up'] = df['return'][df['return'] > 0]
    df['down'] = df['return'][df['return'] < 0]
    df.drop(df.index[1], inplace=True)
    df.fillna(0, inplace=True)
    df = MA(df, 'up', n=5)
    df = MA(df, 'down', n=5)
    df['RSI_'+str(n)] = 100 - 100 / (1 + df['up_MA_5'] / df['down_MA_5'])
    
    return df[['close', 'volume', 'RSI_'+str(n)]]

In [7]:
RSI(df, col='close', n=5)

,close,volume,RSI_5
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
2015-06-08,133.70,53624730,278.688525
...,...,...,...
2020-05-22,330.48,20958094,186.257563
2020-05-26,329.79,32870796,920.769231
2020-05-27,329.09,28621405,640.106952


#### ROC - 5 day

In [8]:
def ROC(df, col='close', n=5):
    '''Calculate an n-period ROC indicator of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col -  specified column to calculate ROC
    n - ROC period
    
    Returns:
    df - dataframe with ROC column
    '''
    df = df.copy()
    df['ROC_'+str(n)] = round(((df[col] - df[col].shift(n)) / df[col].shift(n)) * 100, 4)
    
    return df[['close', 'volume', 'ROC_'+str(n)]]

In [9]:
ROC(df, col='close', n=5)

,close,volume,ROC_5
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
...,...,...,...
2020-05-22,330.48,20958094,3.6280
2020-05-26,329.79,32870796,0.3958
2020-05-27,329.09,28621405,0.5715


#### STOCH - 5 %k days, 3 %d days

In [10]:
def Stoch_Osc(df, col='close', k=5, d=3):
    '''Calculate a k-period slow Stochastic Oscillator indicator and a d-period fast Stochastic Oscillator indicator of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col - specified column to calculate Stochastic Oscillator
    k - slow Stochastic Oscillator period
    d - fast Stochastic Oscillator period
    
    Returns:
    df - dataframe with slow (%k) and fast (%d) Stochastic Oscillator columns
    '''
    df = df.copy()
    df['stoch_%k'] = (df[col] - df[col].rolling(k).min()) / (df[col].rolling(k).max() - df[col].rolling(k).min())
    df = df[['close', 'volume', 'stoch_%k']]
    df = MA(df, 'stoch_%k', n=d)
    df.columns = ['close', 'volume', 'stoch_%k_'+str(k), 'stoch_%d_'+str(d)]
    
    return df

In [11]:
Stoch_Osc(df, col='close', k=5, d=3)

,close,volume,stoch_%k_5,stoch_%d_3
date,,,,
2015-06-01,133.31,33166322,NaN,NaN
2015-06-02,131.26,35078387,NaN,NaN
2015-06-03,135.94,32034911,NaN,NaN
2015-06-04,131.78,38463484,NaN,NaN
2015-06-05,131.58,37314004,0.068376,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,1.000000,0.530908
2020-05-26,329.79,32870796,0.942356,0.647452
2020-05-27,329.09,28621405,0.883876,0.942077


#### MACD - 5/10 short days, 20/40 long days, 5 signal days

In [12]:
def MACD(df, col='close', long=20, short=5, signal=5):
    '''Calculate the distance between a MACD indicator and its signal line of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col - specified column to calculate MACD and its signal line
    short - short MACD period 
    long - long MACD period
    signal - signal line period
    
    Returns:
    df - dataframe with the distance between MACD and signal line column
    '''
    df = df.copy()
    df = MA(df, col='close', n=short)
    df = MA(df, col='close', n=long)
    df['MACD'] = df[str(col)+'_MA_'+str(long)] - df[str(col)+'_MA_'+str(short)]
    df = MA(df, col='MACD', n=signal)
    df['MACD_diff_'+str(long)+'-'+str(short)+'-'+str(signal)] = df['MACD'] - df['MACD_MA_'+str(signal)]
    
    return df[['close', 'volume', 'MACD_diff_'+str(long)+'-'+str(short)+'-'+str(signal)]]

In [13]:
MACD(df, col='close', short=5, long=20, signal=5)

,close,volume,MACD_diff_20-5-5
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
...,...,...,...
2020-05-22,330.48,20958094,-0.3571
2020-05-26,329.79,32870796,1.1180
2020-05-27,329.09,28621405,2.3456


In [14]:
MACD(df, col='close', short=10, long=40, signal=5)

,close,volume,MACD_diff_40-10-5
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
...,...,...,...
2020-05-22,330.48,20958094,1.30270
2020-05-26,329.79,32870796,2.27805
2020-05-27,329.09,28621405,1.79010


#### MOM - 5/20 days

In [15]:
def Momentum(df, col='close', n=5):
    '''Calculate an n-period Momentum indicator of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col - specified column to calculate Momentum
    n - Momentum period
    
    Returns:
    df - dataframe with Momentum column
    '''
    df = df.copy()
    df['momentum_'+str(n)] = df[col] - df[col].shift(n)
    
    return df[['close', 'volume', 'momentum_'+str(n)]]

In [16]:
Momentum(df, col='close', n=5)

,close,volume,momentum_5
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
...,...,...,...
2020-05-22,330.48,20958094,11.57
2020-05-26,329.79,32870796,1.30
2020-05-27,329.09,28621405,1.87


In [17]:
Momentum(df, col='close', n=20)

,close,volume,momentum_20
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,NaN
...,...,...,...
2020-05-22,330.48,20958094,42.24
2020-05-26,329.79,32870796,38.74
2020-05-27,329.09,28621405,42.14


#### VOSC - 2, 5 days

In [18]:
def Vol_Osc(df, col='volume', slow=5, fast=2):
    '''Calculate an n-period Volume Oscillator indicator of the specified column in a dataframe
    Args:
    df - dataframe with specified column
    col - specified column to calculate Volume Oscillator
    n - Volume Oscillator period
    
    Returns:
    df - dataframe with Volume Oscillator column
    '''
    df = df.copy()
    df = MA(df, col=col, n=slow)
    df = MA(df, col=col, n=fast)
    df['vol_'+str(slow)+'-'+str(fast)] = df[str(col)+'_MA_'+str(slow)] - df[str(col)+'_MA_'+str(fast)]
    
    return df[['close', 'volume', 'vol_'+str(slow)+'-'+str(fast)]]

In [19]:
Vol_Osc(df, col='volume', slow=5, fast=2)

,close,volume,vol_5-2
date,,,
2015-06-01,133.31,33166322,NaN
2015-06-02,131.26,35078387,NaN
2015-06-03,135.94,32034911,NaN
2015-06-04,131.78,38463484,NaN
2015-06-05,131.58,37314004,-2677322.4
...,...,...,...
2020-05-22,330.48,20958094,3808134.0
2020-05-26,329.79,32870796,116661.8
2020-05-27,329.09,28621405,-3249324.3


#### All Indicators

In [23]:
pd.merge(RSI(df, col='close', n=5), ROC(df, col='close', n=5), on=['date', 'close', 'volume'])

,close,volume,RSI_5,ROC_5
date,,,,
2015-06-01,133.31,33166322,NaN,NaN
2015-06-03,135.94,32034911,NaN,NaN
2015-06-04,131.78,38463484,NaN,NaN
2015-06-05,131.58,37314004,NaN,NaN
2015-06-08,133.70,53624730,278.688525,0.2926
...,...,...,...,...
2020-05-22,330.48,20958094,186.257563,3.6280
2020-05-26,329.79,32870796,920.769231,0.3958
2020-05-27,329.09,28621405,640.106952,0.5715


In [29]:
indicator_lst = [ROC(df, col='close', n=5), Stoch_Osc(df, col='close', k=5, d=3), 
                 MACD(df, col='close', short=5, long=20, signal=5), Momentum(df, col='close', n=5), 
                 Vol_Osc(df, col='volume', slow=5, fast=2)]

df_ind = RSI(df, col='close', n=5)
for ind in indicator_lst:
    df_ind = pd.merge(df_ind, ind, on = ['date', 'close', 'volume'])
df_ind

,close,volume,RSI_5,ROC_5,stoch_%k_5,stoch_%d_3,MACD_diff_20-5-5,momentum_5,vol_5-2
date,,,,,,,,,
2015-06-01,133.31,33166322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-03,135.94,32034911,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-04,131.78,38463484,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-05,131.58,37314004,NaN,NaN,0.068376,NaN,NaN,NaN,-2677322.4
2015-06-08,133.70,53624730,278.688525,0.2926,0.521368,NaN,NaN,0.39,-6166263.8
...,...,...,...,...,...,...,...,...,...
2020-05-22,330.48,20958094,186.257563,3.6280,1.000000,0.530908,-0.3571,11.57,3808134.0
2020-05-26,329.79,32870796,920.769231,0.3958,0.942356,0.647452,1.1180,1.30,116661.8
2020-05-27,329.09,28621405,640.106952,0.5715,0.883876,0.942077,2.3456,1.87,-3249324.3
